# DQN

In [23]:
import gym
import random
import torch

## 환경

CartPole by OpenAI Gym
* 카트를 잘 밀어서 막대가 넘어지지 않도록 균형을 잡는 문제 
* 액션 : 왼쪽으로 밀기, 오른쪽으로 밀기
* 스텝마다 +1 의 보상을 받기 때문에 보상을 최적화하는 것은 오래도록 균형을 잡는 것을 의미함
* 카트의 상태 s=(카트의 위치, 카트의 속도, 막대의 각도, 막대의 각속도)

In [24]:
import gym

#env=gym.make('CartPole-v1', render_mode="human") # 또는 render_mode="rgb_array"
env=gym.make('CartPole-v1')

for i in range(20):
    observation=env.reset()
    for t in range(100):
        # env.render() # 화면 출력
        action= env.action_space.sample() # action을 랜덤으로 선택
        observation, reward, done, truncated, info = env.step(action)
        
        if done:
            print("{} 번째 episode : {} timestep 뒤에 에피소드가 끝났습니다.".format(i+1, t+1))
            break

1 번째 episode : 24 timestep 뒤에 에피소드가 끝났습니다.
2 번째 episode : 63 timestep 뒤에 에피소드가 끝났습니다.
3 번째 episode : 22 timestep 뒤에 에피소드가 끝났습니다.
4 번째 episode : 32 timestep 뒤에 에피소드가 끝났습니다.
5 번째 episode : 14 timestep 뒤에 에피소드가 끝났습니다.
6 번째 episode : 10 timestep 뒤에 에피소드가 끝났습니다.
7 번째 episode : 48 timestep 뒤에 에피소드가 끝났습니다.
8 번째 episode : 18 timestep 뒤에 에피소드가 끝났습니다.
9 번째 episode : 26 timestep 뒤에 에피소드가 끝났습니다.
10 번째 episode : 85 timestep 뒤에 에피소드가 끝났습니다.
11 번째 episode : 28 timestep 뒤에 에피소드가 끝났습니다.
12 번째 episode : 13 timestep 뒤에 에피소드가 끝났습니다.
13 번째 episode : 35 timestep 뒤에 에피소드가 끝났습니다.
14 번째 episode : 24 timestep 뒤에 에피소드가 끝났습니다.
15 번째 episode : 12 timestep 뒤에 에피소드가 끝났습니다.
16 번째 episode : 11 timestep 뒤에 에피소드가 끝났습니다.
17 번째 episode : 21 timestep 뒤에 에피소드가 끝났습니다.
18 번째 episode : 33 timestep 뒤에 에피소드가 끝났습니다.
19 번째 episode : 34 timestep 뒤에 에피소드가 끝났습니다.
20 번째 episode : 13 timestep 뒤에 에피소드가 끝났습니다.


## Replay Buffer

최신 5만 개의 데이터를 들고 있다가 필요할 때마다 batch_size 만큼의 데이터를 뽑아서 제공

In [25]:
import collections # replay buffer을 구현하기 위함 -> deque의 FIFO를 이용

In [26]:
# Hyperparameters
buffer_limit = 50000

class ReplayBuffer():
    def __init__(self):
        self.buffer = collections.deque(maxlen=buffer_limit)

    def put(self, transition): # 데이터를 buffer에 저장
        self.buffer.append(transition)

    def sample(self, n): # 버퍼에서 랜덤하게 buffer_size 만큼의 데이터를 뽑아서 미니 배치를 구성해주는 함수
        mini_batch = random.sample(self.buffer, n)
        s_lst, a_lst, r_lst, s_prime_lst, done_mask_lst = [], [], [], [], []
        for transition in mini_batch:
            s, a, r, s_prime, done_mask = transition
            s_lst.append(s)
            a_lst.append([a])
            r_lst.append([r])
            s_prime_lst.append(s_prime)
            done_mask_lst.append([done_mask])
        
        # 각각의 데이터를 tensor로 변환
        return torch.tensor(s_lst, dtype=torch.float), torch.tensor(a_lst), \
            torch.tensor(r_lst), torch.tensor(s_prime_lst, dtype=torch.float), \
            torch.tensor(done_mask_lst)
    
    def size(self):
        return len(self.buffer)

## 에이전트

In [27]:
import torch.nn as nn
import torch.nn.functional as F

In [28]:
class Qnet(nn.Module):
    def __init__(self):
        super(Qnet, self).__init__()
        self.fc1 = nn.Linear(4, 128) # input 차원 : state 4개
        self.fc2 = nn.Linear(128, 128)
        self.fc3 = nn.Linear(128, 2) # output 차원 : action 2개
    
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x) # 마지막 layer에서는 activation function을 사용하지 않음
        return x
    
    def sample_action(self, obs, epsilon): # epsilon greedy 방식으로 action을 선택
        out = self.forward(obs)
        coin = random.random()
        if coin < epsilon:
            return random.randint(0, 1)
        else:
            return out.argmax().item()

### 학습함수

* 한 episode가 끝날 때마다 총 320 개의 데이터를 뽑아서 사용
* 별도의 Target network를 두었음 : q_target

In [29]:
# Hyperparameters
gamma = 0.98
batch_size=32

In [30]:
def train(q, q_target, memory, optimizer):
    for i in range(10): # 10 개의 mini-batch 뽑아서 학습
        s, a, r, s_prime, done_mask = memory.sample(batch_size)

        q_out = q(s) # input : state
        q_a = q_out.gather(1, a) # 실제 선택된 액션의 q값
        max_q_prime = q_target(s_prime).max(1)[0].unsqueeze(1)
        target = r + gamma * max_q_prime * done_mask # target q값

        # 각 mini-batch마다 loss 계산
        loss = F.smooth_l1_loss(q_a, target)

        optimizer.zero_grad()
        loss.backward() # gradient 계산
        optimizer.step() # qnet의 파라미터 업데이트

## 메인함수

In [31]:
# Hyperparameters
learning_rate = 0.0005

In [32]:
import torch.optim as optim

In [33]:
from gym.wrappers.record_video import RecordVideo

In [59]:
def main():
    env = gym.make('CartPole-v1', render_mode="rgb_array")

    #env = RecordVideo(env, './video', episode_trigger= lambda episode_number: episode_number%100==0)
    #s, _ = env.reset()
    #env.start_video_recorder()

    # Q network와 Q target network를 생성
    q = Qnet()
    q_target = Qnet()
    q_target.load_state_dict(q.state_dict())

    # replay buffer 생성
    memory = ReplayBuffer()

    print_interval = 20
    score = 0.0
    sum_score = 0.0
    optimizer = optim.Adam(q.parameters(), lr=learning_rate)

    for n_epi in range(600):
        epsilon = max(0.01, 0.08 - 0.01*(n_epi/200)) # Linear annealing from 8% to 1%
        s, _ = env.reset() ## 수정
        done = False

        # 데이터 쌓기
        while not done:
            a = q.sample_action(torch.from_numpy(s).float(), epsilon)
            s_prime, r, done, truncated, info = env.step(a) ## 수정
            done_mask = 0.0 if done else 1.0
            memory.put((s, a, r/100.0, s_prime, done_mask)) # reward scaling : 100으로 나눠줌
            s = s_prime
            score += r

            if done:
                break

            if score > 50000: 
                print("Wow! Score is over 50000!")
            
        sum_score += score
        score = 0.0

        # 학습하기
        if memory.size() > 2000: # 리플레이 버퍼에 데이터가 충분히 쌓이지 않았을 때 학습을 진행하면 초기의 데이터가 많이 재사용되어 학습이 치우쳐짐
            train(q, q_target, memory, optimizer) # episode가 한번 끝날 때마다 train 함수를 호출하여 NN 학습 (q_target network는 업데이트하지 않음)

        # 출력하기
        if n_epi%print_interval==0 and n_epi!=0:
            q_target.load_state_dict(q.state_dict()) # 20번의 episode마다 q_target network를 업데이트
            print("# of episode :{}, avg score : {:.1f}, buffer size : {}, epsilon : {:.1f}%"
                  .format(n_epi, sum_score/print_interval, memory.size(), epsilon*100))
            sum_score = 0.0

    #env.close_video_recorder()
    env.close()

In [60]:
if __name__ == '__main__':
    main()

# of episode :20, avg score : 42.4, buffer size : 848, epsilon : 7.9%
# of episode :40, avg score : 43.8, buffer size : 1724, epsilon : 7.8%
# of episode :60, avg score : 24.6, buffer size : 2216, epsilon : 7.7%
# of episode :80, avg score : 13.8, buffer size : 2492, epsilon : 7.6%
# of episode :100, avg score : 11.8, buffer size : 2728, epsilon : 7.5%
# of episode :120, avg score : 40.1, buffer size : 3531, epsilon : 7.4%
# of episode :140, avg score : 24.5, buffer size : 4021, epsilon : 7.3%
# of episode :160, avg score : 15.1, buffer size : 4322, epsilon : 7.2%
# of episode :180, avg score : 12.8, buffer size : 4578, epsilon : 7.1%
# of episode :200, avg score : 10.3, buffer size : 4784, epsilon : 7.0%
# of episode :220, avg score : 12.2, buffer size : 5029, epsilon : 6.9%
# of episode :240, avg score : 31.6, buffer size : 5660, epsilon : 6.8%
# of episode :260, avg score : 88.6, buffer size : 7432, epsilon : 6.7%
# of episode :280, avg score : 149.4, buffer size : 10420, epsilon : 

## 비디오 확인

In [36]:
# 결과를 mp4 동영상으로 보여주기 위한 코드
import glob
import io
import base64
from IPython.display import HTML
from IPython import display as ipythondisplay

def show_video():
    mp4list = glob.glob('video/*.mp4')
    if len(mp4list) > 0:
        mp4 = mp4list[0]
        video = io.open(mp4, 'r+b').read()
        encoded = base64.b64encode(video)
        ipythondisplay.display(HTML(data='''<video alt="test" autoplay 
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
                </video>'''.format(encoded.decode('ascii'))))
    else: 
        print("Could not find video")

from pyvirtualdisplay import Display
display = Display(visible=0, size=(1400, 900))
display.start()

In [37]:
show_video()

Could not find video
